In [15]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import pandas as pd
from Bio.Blast.Applications import NcbipsiblastCommandline
from Bio.Blast import NCBIXML
type = "H1N1"
from Bio import SeqIO

In [4]:

# oriH = pd.read_excel(f'data/{type}/{type}_all.xlsx')
# oriH["year"] = oriH["time"].dt.year
# minY = oriH["year"].min()
# maxY = oriH["year"].max()
# print(minY,maxY)
# allYArr = {}
# for i in range(minY,maxY+1):
#     allYArr[i] = []
# my_records  = []
# for i, j in oriH.iterrows():
#     id = j["strain"].replace(" ","_")
#     allYArr[j["year"]].append(SeqRecord(Seq(j["seq"]),id=id,description=str(j["time"])))
#
# from Bio import SeqIO
# # for key in allYArr:
# #     SeqIO.write(allYArr[key], f"{type}_{key}.fasta", "fasta")
# SeqIO.write(allYArr[2022], f"{type}_{2022}.fasta", "fasta")

1991 2022


26874

In [7]:
# from Bio.Blast.Applications import NcbipsiblastCommandline
# y = 1999
# blastPlant = f"D:\\blast-BLAST_VERSION+\\bin\\psiblast.exe"
# db = f"D:\\blast-BLAST_VERSION+\database\H1N1\H1N1{y}DB"
# blastn_cline = NcbipsiblastCommandline(cmd=blastPlant,query="t1emp.fasta", db=db, evalue=0.00001,outfmt=5, out="opuntia.xml",max_target_seqs=5)
#
# print(blastn_cline())
# result_handle = open("opuntia.xml")
# from Bio.Blast import NCBIXML
# blast_records = NCBIXML.parse(result_handle)
# for blast_record in blast_records:
#     for alignment in blast_record.alignments:
#         print ('name:', alignment.title)
#         print ('name:', alignment.hsps[0].sbjct)
# result_handle.close()

In [16]:
allData =pd.read_excel(f'data/{type}/{type}_all.xlsx')
# allData["strain"].replace(r'\s+','_',regex=True,inplace=True)
allData["strain"]

0              >A/Belgium/S00046/2017
1             >A/La_Serena/76388/2015
2              >A/Santiago/77846/2015
3                ?/AstanaZVL/381/2018
4               ?/Krasnoyarsk/61/2019
                     ...             
114266    _A/Ankara/5607/2016_(TR-26)
114267          _A/Athens.GR/167/2015
114268          _A/Osoorno/53068/2022
114269         _A/Romania/544062/2022
114270               a/Paris/798/2014
Name: strain, Length: 114271, dtype: object

In [17]:
from collections import Counter
time = allData["time"].dt.year

timeNum = dict(Counter(time))
timeNum

{2017: 15650,
 2015: 6222,
 2018: 9535,
 2019: 13701,
 2016: 8172,
 2022: 26874,
 2014: 4990,
 2006: 1024,
 2005: 909,
 2013: 2732,
 2009: 1466,
 2011: 2188,
 2004: 917,
 2008: 1616,
 2012: 3644,
 2020: 2359,
 2010: 1530,
 2021: 5386,
 2003: 1085,
 2002: 650,
 1994: 130,
 2007: 1237,
 1993: 208,
 1995: 153,
 2001: 264,
 1992: 114,
 1999: 407,
 1996: 204,
 2000: 395,
 1998: 246,
 1997: 187,
 1991: 76}

In [18]:
rs = pd.read_excel(f"data/{type}/{type}_preDom.xlsx")
from collections import Counter
time = rs["ReportYear"]

timeNum = dict(Counter(time))
timeNum

{2003: 11,
 2004: 3,
 2005: 25,
 2006: 14,
 2007: 17,
 2008: 18,
 2009: 51,
 2010: 45,
 2011: 226,
 2012: 882,
 2013: 318,
 2014: 685,
 2015: 411,
 2016: 92,
 2017: 303,
 2018: 81,
 2019: 103,
 2020: 64,
 2021: 49,
 2022: 1035}

In [19]:
import math

blastPlant = f"D:\\App\\blast-BLAST_VERSION+\\bin\\blastp.exe"
seqNum = 1000
rsv = rs.values
arrY = {}
labelArr = []
for i in range(2003,2023):
    arrY[i] = []
for r in rsv:
    strain = r[0]
    year = r[2]
    if year<1999 :continue
    seq = r[4]
    arrY[year].append(SeqRecord(Seq(seq),id=strain,description=str(year)))
    labelArr.append([strain,seq,year])
for key in range(2003,2023):
    SeqIO.write(arrY[key], "temp.fasta", "fasta")
    db = f"D:\\App\\blast-BLAST_VERSION+\database\\{type}\\{type}{key}DB"
    max_sI = 0
    ms = seqNum-timeNum[key]
    if ms < 0:
        continue
    else:
        ms = math.ceil(ms/timeNum[key]) 
    blastn_cline = NcbipsiblastCommandline(cmd=blastPlant,query="temp.fasta", db=db, evalue=0.0001,outfmt=5, out="opuntia.xml",max_target_seqs=ms)
    print(blastn_cline())
    result_handle = open("opuntia.xml")
    blast_records = NCBIXML.parse(result_handle)
    for blast_record in blast_records:
        for alignment in blast_record.alignments:
            labelArr.append([alignment.hit_def.split(" ")[0],alignment.hsps[0].sbjct,key])
    result_handle.close()
df = pd.DataFrame(labelArr, columns=['strain', "seq","year"])
df.drop_duplicates(subset=['strain', 'year'])
df["strain"]

('', '')


0       A/Hong_Kong/1550/2002
1            A/Egypt/130/2002
2          A/Finland/170/2003
3          A/Latvia/1506/2003
4          A/Finland/300/2003
                ...          
996        A/England/602/2003
997        A/England/587/2003
998        A/England/909/2003
999        A/England/492/2003
1000       A/England/491/2003
Name: strain, Length: 1001, dtype: object

In [20]:
df = df.sort_values(by='year')

df = df[df['seq'].str.len() == 566]

df = df.reset_index(drop=True)


In [59]:
df.to_excel(f"data/{type}/{type}_labelData.xlsx",index=False)

In [21]:
seqs_len = 15
time_seq = {}
import random
def searchBlast(year,seq_arr):
    SeqIO.write(seq_arr, "temp.fasta", "fasta")
    db = f"D:\\App\\blast-BLAST_VERSION+\database\\{type}\\{type}{year}DB"
    blastn_cline = NcbipsiblastCommandline(cmd=blastPlant,query="temp.fasta", db=db, evalue=0.0001,outfmt=5,num_threads=8, out="temp.xml",max_target_seqs=5)
    print(blastn_cline())
    result_handle = open("temp.xml")

    return result_handle
def addseq(strain,year,seq):
    if strain not in time_seq:
        time_seq[strain] = {
            "year":year,
            "seq":seq,
            "preSeq":[]
        }
        return True
    else:
        return False
for i in range(2003,2023):
    search_seq = df[df["year"]==i]

    seq_temparr = []
    for s in search_seq.values:
        # print(s)
        if addseq(s[0],s[2],s[1]):
            seq_temparr.append(SeqRecord(Seq(s[1]),id=s[0],description=str(s[2])))


    for j in range(1,seqs_len):
        nextY = i-j
        if len(seq_temparr)==0: continue
        # print(seq_temparr)
        result_handle = searchBlast(nextY,seq_temparr)
        blast_records = NCBIXML.parse(result_handle)
        seq_temparr = []
        for blast_record in blast_records:

            strain_name = blast_record.query.split(" ")[0]

            controlNum = random.randint(1,3)
            # sn = 0
            for alignment in blast_record.alignments:


                hit_strain = alignment.hit_def.split(" ")[0]
                hit_seq = alignment.hsps[0].sbjct
                time_seq[strain_name]["preSeq"].append(hit_strain)

                if addseq(hit_strain,nextY,hit_seq):
                    seq_temparr.append(SeqRecord(Seq(hit_seq),id=hit_strain,description=str(nextY)))

                else:
                    continue
            # print(seq_temparr)
        result_handle.close()




('', '')
('', '')
('', '')
('', '')
('', '')


In [22]:
df1 = pd.DataFrame(time_seq)
df1 = df1.T
df1.index.name = 'strain'
df1.reset_index(inplace=True)
# df1 = pd.DataFrame(df1.values.T,columns=["strain","year","seq","pre"])
# pd.melt(df1,var_name=["strain","year","seq","pre"])




df1.to_excel(f"{type}_data.xlsx",index=False)

In [23]:

df1 = pd.read_excel(f"{type}_data.xlsx")

In [25]:
preArr = df1["preSeq"].values
from ast import literal_eval
preArr



array(['A/New_York/256/1998,A/New_York/304/1998,A/New_York/249/1998,A/New_York/240/1998,A/New_York/254/1998',
       'A/New_York/256/1998,A/New_York/304/1998,A/New_York/249/1998,A/New_York/224/1998,A/New_York/240/1998',
       'A/New_York/256/1998,A/New_York/304/1998,A/New_York/249/1998,A/New_York/254/1998,A/New_York/224/1998',
       'A/New_York/256/1998,A/New_York/304/1998,A/New_York/249/1998,A/New_York/254/1998,A/New_York/224/1998',
       'A/New_York/256/1998,A/New_York/304/1998,A/New_York/249/1998,A/New_York/254/1998,A/New_York/224/1998',
       'A/New_York/256/1998,A/New_York/304/1998,A/New_York/249/1998,A/New_York/254/1998,A/New_York/224/1998',
       'A/New_York/256/1998,A/New_York/304/1998,A/New_York/249/1998,A/New_York/254/1998,A/New_York/224/1998',
       'A/New_York/256/1998,A/New_York/304/1998,A/New_York/249/1998,A/New_York/224/1998,A/New_York/313/1998',
       'A/New_York/256/1998,A/New_York/304/1998,A/New_York/249/1998,A/New_York/224/1998,A/New_York/313/1998',
       'A/

In [26]:
pre = [[],[],[],[],[]]

for pres in preArr:
    # pres = literal_eval(pres)
    pres = str(pres).split(",")
    seq = allData[allData["strain"].isin(pres)]["seq"]
    if len(seq)==5:

        for i in range(5):
            pre[i].append(seq.values[i])
    else:
        if len(seq)>0:
            pre[0].append(seq.values[0])
        else:
            pre[0].append("")
        for i in range(1,5):
            pre[i].append("")

df1["s1"] = pre[0]
df1["s2"] = pre[1]
df1["s3"] = pre[2]
df1["s4"] = pre[3]
df1["s5"] = pre[4]

In [27]:

df1.to_excel(f"{type}_data.xlsx",index=False)

In [52]:
# dfl = pd.read_excel(f"data/{type}/{type}_labelData.xlsx")
# dfl = dfl.merge(df1, left_on="strain", right_on="strain", how="left")
# # dfl.drop(['cyear'], axis=1, inplace=True)
# dfl.to_excel(f"data/{type}/{type}_data.xlsx",index=False)